In [1]:
import keras
import numpy as np

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [18]:
np.random.seed(123)

## Preprocessing Data

In [2]:
def get_mnist_data():
    from keras.datasets import mnist
    from keras.utils import np_utils
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1], X_train.shape[2])
    y_train = y_train.reshape(y_train.shape[0], 1)
    X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1], X_test.shape[2])
    y_test = y_test.reshape(y_test.shape[0], 1)
    
    y_train = np_utils.to_categorical(y_train)
    y_test  = np_utils.to_categorical(y_test)
    
    return X_train, y_train, X_test, y_test

In [3]:
X_train, y_train, X_test, y_test = get_mnist_data()

In [4]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 1, 28, 28), (60000, 10), (10000, 1, 28, 28), (10000, 10))

In [5]:
X_mean = X_train.mean().astype(np.float32)

In [6]:
X_std = X_train.std().astype(np.float32)

In [7]:
def normalizer(x):
    return (x - X_mean) / X_std

## Building the model

In [8]:
from keras.layers import Convolution2D, Dense, Flatten, Lambda, Dropout
from keras.models import Sequential
from keras.optimizers import Adam

### Neural Networks Model

In [20]:
nb_epoch = 5

In [14]:
model_1 = Sequential()
model_1.add(Lambda(normalizer, input_shape=(1, 28, 28)))
model_1.add(Flatten())
model_1.add(Dense(512, activation='softmax'))
model_1.add(Dense(10, activation='softmax'))

In [15]:
model_1.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'] )

In [25]:
model_1.fit(X_train, y_train,batch_size=64, validation_data=(X_test, y_test), nb_epoch=nb_epoch)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 2s - loss: 0.2375 - acc: 0.9331 - val_loss: 0.2949 - val_acc: 0.9257
Epoch 2/5
60000/60000 [==============================] - 2s - loss: 0.2378 - acc: 0.9332 - val_loss: 0.3009 - val_acc: 0.9213
Epoch 3/5
60000/60000 [==============================] - 2s - loss: 0.2384 - acc: 0.9335 - val_loss: 0.2959 - val_acc: 0.9224
Epoch 4/5
60000/60000 [==============================] - 2s - loss: 0.2370 - acc: 0.9342 - val_loss: 0.2972 - val_acc: 0.9252
Epoch 5/5
60000/60000 [==============================] - 2s - loss: 0.2385 - acc: 0.9335 - val_loss: 0.3060 - val_acc: 0.9212


### Is learning rate optimal?

default in Adam is 0.001

In [19]:
model_1.optimizer.lr = 0.1

In [23]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s - loss: 0.2359 - acc: 0.9344 - val_loss: 0.2952 - val_acc: 0.9233
Epoch 2/10
60000/60000 [==============================] - 2s - loss: 0.2368 - acc: 0.9344 - val_loss: 0.2924 - val_acc: 0.9250
Epoch 3/10
60000/60000 [==============================] - 2s - loss: 0.2393 - acc: 0.9338 - val_loss: 0.2980 - val_acc: 0.9241
Epoch 4/10
60000/60000 [==============================] - 2s - loss: 0.2384 - acc: 0.9338 - val_loss: 0.3030 - val_acc: 0.9186
Epoch 5/10
60000/60000 [==============================] - 2s - loss: 0.2390 - acc: 0.9328 - val_loss: 0.3011 - val_acc: 0.9220
Epoch 6/10
60000/60000 [==============================] - 2s - loss: 0.2389 - acc: 0.9330 - val_loss: 0.2949 - val_acc: 0.9248
Epoch 7/10
60000/60000 [==============================] - 2s - loss: 0.2376 - acc: 0.9336 - val_loss: 0.2973 - val_acc: 0.9207
Epoch 8/10
60000/60000 [==============================] - 2s 

** Interesting **

Appears like batch size also affects the performance. Why is this? Look into Adam more. If it is similar to SGD, this makes sense.

Effects of batch_size:
* Higher ==> less time to train
* Seems to affect the performance as well. So, you could consider this as a hyperparameter as well. 

In [26]:
model_1.optimizer.lr = 0.0001

In [28]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=20)

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 2s - loss: 0.2359 - acc: 0.9338 - val_loss: 0.3044 - val_acc: 0.9207
Epoch 2/20
60000/60000 [==============================] - 2s - loss: 0.2364 - acc: 0.9337 - val_loss: 0.3037 - val_acc: 0.9221
Epoch 3/20
60000/60000 [==============================] - 2s - loss: 0.2361 - acc: 0.9328 - val_loss: 0.3002 - val_acc: 0.9236
Epoch 4/20
60000/60000 [==============================] - 2s - loss: 0.2358 - acc: 0.9342 - val_loss: 0.3106 - val_acc: 0.9209
Epoch 5/20
60000/60000 [==============================] - 2s - loss: 0.2367 - acc: 0.9341 - val_loss: 0.3038 - val_acc: 0.9198
Epoch 6/20
60000/60000 [==============================] - 2s - loss: 0.2357 - acc: 0.9339 - val_loss: 0.2996 - val_acc: 0.9237
Epoch 7/20
60000/60000 [==============================] - 2s - loss: 0.2362 - acc: 0.9336 - val_loss: 0.3017 - val_acc: 0.9240
Epoch 8/20
60000/60000 [==============================] - 2s 

### Learning Rate and its usage

We could try the following
* Start with a relatively high learning rate for a few epochs
* Reduce it and continue for a few more epochs
* Reduce it further and continue for more epochs(make sure that it doesn't go so far that it overfits)

In [39]:
# Let's put this in action and do it right this time!
model_1 = Sequential()
model_1.add(Lambda(normalizer, input_shape=(1, 28, 28)))
model_1.add(Flatten())
model_1.add(Dense(512, activation='softmax'))
model_1.add(Dense(10, activation='softmax'))
model_1.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'] )

In [40]:
model_1.optimizer.lr = 0.1

In [41]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s - loss: 1.1138 - acc: 0.6543 - val_loss: 0.9152 - val_acc: 0.7103
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.8796 - acc: 0.7281 - val_loss: 0.9235 - val_acc: 0.6600
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 0.8426 - acc: 0.7354 - val_loss: 0.7523 - val_acc: 0.7699
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 0.8255 - acc: 0.7453 - val_loss: 0.7188 - val_acc: 0.7746
Epoch 5/5
60000/60000 [==============================] - 4s - loss: 0.8032 - acc: 0.7579 - val_loss: 0.8076 - val_acc: 0.7681


Hmmm, let's try more epochs with same learning rate and see where it gets us

In [42]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=8)

Train on 60000 samples, validate on 10000 samples
Epoch 1/8
60000/60000 [==============================] - 4s - loss: 0.7954 - acc: 0.7589 - val_loss: 0.8208 - val_acc: 0.7361
Epoch 2/8
60000/60000 [==============================] - 4s - loss: 0.8315 - acc: 0.7377 - val_loss: 0.8279 - val_acc: 0.7601
Epoch 3/8
60000/60000 [==============================] - 4s - loss: 0.8101 - acc: 0.7524 - val_loss: 0.9932 - val_acc: 0.6271
Epoch 4/8
60000/60000 [==============================] - 4s - loss: 0.7839 - acc: 0.7536 - val_loss: 0.7182 - val_acc: 0.7758
Epoch 5/8
60000/60000 [==============================] - 4s - loss: 0.7560 - acc: 0.7674 - val_loss: 0.7024 - val_acc: 0.7941
Epoch 6/8
60000/60000 [==============================] - 4s - loss: 0.7387 - acc: 0.7756 - val_loss: 0.7308 - val_acc: 0.7848
Epoch 7/8
60000/60000 [==============================] - 4s - loss: 0.7494 - acc: 0.7664 - val_loss: 0.8653 - val_acc: 0.7374
Epoch 8/8
60000/60000 [==============================] - 4s - loss: 

Clearly, the learning rate is too much and it's not moving from a rut that it had ended up at

In [43]:
model_1.optimizer.lr = 0.001

In [44]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=8)

Train on 60000 samples, validate on 10000 samples
Epoch 1/8
60000/60000 [==============================] - 4s - loss: 0.7346 - acc: 0.7761 - val_loss: 0.7364 - val_acc: 0.7667
Epoch 2/8
60000/60000 [==============================] - 4s - loss: 0.7351 - acc: 0.7767 - val_loss: 0.7239 - val_acc: 0.7704
Epoch 3/8
60000/60000 [==============================] - 4s - loss: 0.7265 - acc: 0.7771 - val_loss: 0.7133 - val_acc: 0.7926
Epoch 4/8
60000/60000 [==============================] - 4s - loss: 0.7202 - acc: 0.7816 - val_loss: 0.6639 - val_acc: 0.8006
Epoch 5/8
60000/60000 [==============================] - 4s - loss: 0.6918 - acc: 0.7904 - val_loss: 0.6952 - val_acc: 0.7875
Epoch 6/8
60000/60000 [==============================] - 4s - loss: 0.7651 - acc: 0.7555 - val_loss: 0.8098 - val_acc: 0.7260
Epoch 7/8
60000/60000 [==============================] - 4s - loss: 0.7216 - acc: 0.7742 - val_loss: 0.6543 - val_acc: 0.7994
Epoch 8/8
60000/60000 [==============================] - 4s - loss: 

In [45]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s - loss: 0.6931 - acc: 0.7927 - val_loss: 0.6651 - val_acc: 0.7925
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.7213 - acc: 0.7792 - val_loss: 0.6948 - val_acc: 0.7833
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 0.7358 - acc: 0.7741 - val_loss: 0.7469 - val_acc: 0.7698
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 0.7124 - acc: 0.7815 - val_loss: 0.6747 - val_acc: 0.7905
Epoch 5/5
60000/60000 [==============================] - 4s - loss: 0.7139 - acc: 0.7762 - val_loss: 0.7484 - val_acc: 0.7765


In [46]:
model_1.optimizer.lr = 0.0001

In [47]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=15)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 4s - loss: 0.7012 - acc: 0.7798 - val_loss: 0.6365 - val_acc: 0.7980
Epoch 2/15
60000/60000 [==============================] - 4s - loss: 0.6700 - acc: 0.7938 - val_loss: 0.6795 - val_acc: 0.7966
Epoch 3/15
60000/60000 [==============================] - 4s - loss: 0.6745 - acc: 0.7950 - val_loss: 0.6026 - val_acc: 0.8094
Epoch 4/15
60000/60000 [==============================] - 4s - loss: 0.6690 - acc: 0.7952 - val_loss: 0.6505 - val_acc: 0.7922
Epoch 5/15
60000/60000 [==============================] - 4s - loss: 0.6685 - acc: 0.7923 - val_loss: 0.6738 - val_acc: 0.7788
Epoch 6/15
60000/60000 [==============================] - 4s - loss: 0.6982 - acc: 0.7869 - val_loss: 0.6788 - val_acc: 0.7870
Epoch 7/15
60000/60000 [==============================] - 4s - loss: 0.6867 - acc: 0.7865 - val_loss: 0.6647 - val_acc: 0.7878
Epoch 8/15
60000/60000 [==============================] - 4s 

In [48]:
model_1.optimizer.lr = 1

In [49]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s - loss: 0.6653 - acc: 0.7962 - val_loss: 0.6289 - val_acc: 0.7976
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.6571 - acc: 0.7983 - val_loss: 0.6479 - val_acc: 0.7945
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 0.6578 - acc: 0.7967 - val_loss: 0.6169 - val_acc: 0.8040
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 0.6512 - acc: 0.8051 - val_loss: 0.6108 - val_acc: 0.8066
Epoch 5/5
60000/60000 [==============================] - 4s - loss: 0.6520 - acc: 0.8058 - val_loss: 0.6664 - val_acc: 0.7875


In [50]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s - loss: 0.6994 - acc: 0.7982 - val_loss: 0.6625 - val_acc: 0.8036
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.6607 - acc: 0.8021 - val_loss: 0.6020 - val_acc: 0.8186
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 0.6419 - acc: 0.8057 - val_loss: 0.5932 - val_acc: 0.8133
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 0.6497 - acc: 0.8010 - val_loss: 0.6344 - val_acc: 0.8093
Epoch 5/5
60000/60000 [==============================] - 4s - loss: 0.7069 - acc: 0.7767 - val_loss: 0.6859 - val_acc: 0.7690


### Nope. We seem to have reached a dead end. A local minima perhaps?

Let's start over!

In [51]:
model_1 = Sequential()
model_1.add(Lambda(normalizer, input_shape=(1, 28, 28)))
model_1.add(Flatten())
model_1.add(Dense(512, activation='softmax'))
model_1.add(Dense(10, activation='softmax'))
model_1.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'] )

In [52]:
# default lr
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s - loss: 1.5699 - acc: 0.8225 - val_loss: 1.0521 - val_acc: 0.8528
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.8081 - acc: 0.8509 - val_loss: 0.6336 - val_acc: 0.8466
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 0.5496 - acc: 0.8542 - val_loss: 0.4904 - val_acc: 0.8549
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 0.4544 - acc: 0.8597 - val_loss: 0.4349 - val_acc: 0.8546
Epoch 5/5
60000/60000 [==============================] - 4s - loss: 0.4090 - acc: 0.8645 - val_loss: 0.4268 - val_acc: 0.8599


already doing much better

Moral of the story - ** Learning rate matters. If shits going wrong initially itself, check lr maybe? **

In [53]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s - loss: 0.3831 - acc: 0.8712 - val_loss: 0.4044 - val_acc: 0.8714
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.3649 - acc: 0.8761 - val_loss: 0.3832 - val_acc: 0.8693
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 0.3503 - acc: 0.8847 - val_loss: 0.3749 - val_acc: 0.8798
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 0.3333 - acc: 0.8990 - val_loss: 0.3583 - val_acc: 0.8879
Epoch 5/5
60000/60000 [==============================] - 4s - loss: 0.3112 - acc: 0.9128 - val_loss: 0.3585 - val_acc: 0.8933


In [54]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s - loss: 0.2893 - acc: 0.9224 - val_loss: 0.3185 - val_acc: 0.9155
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.2751 - acc: 0.9266 - val_loss: 0.3109 - val_acc: 0.9193
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 0.2580 - acc: 0.9315 - val_loss: 0.3257 - val_acc: 0.9086
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 0.2490 - acc: 0.9329 - val_loss: 0.2897 - val_acc: 0.9238
Epoch 5/5
60000/60000 [==============================] - 4s - loss: 0.2380 - acc: 0.9355 - val_loss: 0.2936 - val_acc: 0.9225


In [55]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s - loss: 0.2231 - acc: 0.9413 - val_loss: 0.2589 - val_acc: 0.9349
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.2044 - acc: 0.9466 - val_loss: 0.2482 - val_acc: 0.9349
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 0.1905 - acc: 0.9500 - val_loss: 0.2387 - val_acc: 0.9401
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 0.1808 - acc: 0.9524 - val_loss: 0.2287 - val_acc: 0.9402
Epoch 5/5
60000/60000 [==============================] - 4s - loss: 0.1774 - acc: 0.9525 - val_loss: 0.2273 - val_acc: 0.9376


In [56]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s - loss: 0.1712 - acc: 0.9540 - val_loss: 0.2323 - val_acc: 0.9409
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.1662 - acc: 0.9553 - val_loss: 0.2302 - val_acc: 0.9406
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 0.1612 - acc: 0.9567 - val_loss: 0.2249 - val_acc: 0.9437
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 0.1576 - acc: 0.9581 - val_loss: 0.2298 - val_acc: 0.9417
Epoch 5/5
60000/60000 [==============================] - 4s - loss: 0.1565 - acc: 0.9572 - val_loss: 0.2228 - val_acc: 0.9443


In [57]:
# Starting to overfit now. But let's see where it goes
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s - loss: 0.1495 - acc: 0.9597 - val_loss: 0.2308 - val_acc: 0.9458
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.1475 - acc: 0.9603 - val_loss: 0.2287 - val_acc: 0.9449
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 0.1460 - acc: 0.9609 - val_loss: 0.2322 - val_acc: 0.9437
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 0.1416 - acc: 0.9624 - val_loss: 0.2267 - val_acc: 0.9459
Epoch 5/5
60000/60000 [==============================] - 4s - loss: 0.1405 - acc: 0.9629 - val_loss: 0.2170 - val_acc: 0.9470


In [58]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s - loss: 0.1361 - acc: 0.9638 - val_loss: 0.2243 - val_acc: 0.9436
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.1347 - acc: 0.9641 - val_loss: 0.2483 - val_acc: 0.9393
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 0.1316 - acc: 0.9651 - val_loss: 0.2336 - val_acc: 0.9447
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 0.1301 - acc: 0.9650 - val_loss: 0.2396 - val_acc: 0.9409
Epoch 5/5
60000/60000 [==============================] - 4s - loss: 0.1301 - acc: 0.9652 - val_loss: 0.2287 - val_acc: 0.9447


In [59]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s - loss: 0.1250 - acc: 0.9658 - val_loss: 0.2269 - val_acc: 0.9477
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.1233 - acc: 0.9667 - val_loss: 0.2198 - val_acc: 0.9478
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 0.1226 - acc: 0.9670 - val_loss: 0.2294 - val_acc: 0.9450
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 0.1211 - acc: 0.9681 - val_loss: 0.2205 - val_acc: 0.9480
Epoch 5/5
60000/60000 [==============================] - 4s - loss: 0.1194 - acc: 0.9680 - val_loss: 0.2214 - val_acc: 0.9458


## Seems to be overfitting the data

**Question** 

How much difference in training and validation data is tolerable? How much is too much?

In [60]:
model_1.optimizer.lr = 0.0001

In [61]:
model_1.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=10)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 4s - loss: 0.1182 - acc: 0.9686 - val_loss: 0.2274 - val_acc: 0.9466
Epoch 2/10
60000/60000 [==============================] - 4s - loss: 0.1156 - acc: 0.9691 - val_loss: 0.2204 - val_acc: 0.9480
Epoch 3/10
60000/60000 [==============================] - 4s - loss: 0.1132 - acc: 0.9702 - val_loss: 0.2254 - val_acc: 0.9481
Epoch 4/10
60000/60000 [==============================] - 4s - loss: 0.1139 - acc: 0.9697 - val_loss: 0.2246 - val_acc: 0.9480
Epoch 5/10
60000/60000 [==============================] - 4s - loss: 0.1112 - acc: 0.9699 - val_loss: 0.2201 - val_acc: 0.9478
Epoch 6/10
60000/60000 [==============================] - 4s - loss: 0.1105 - acc: 0.9705 - val_loss: 0.2189 - val_acc: 0.9477
Epoch 7/10
60000/60000 [==============================] - 4s - loss: 0.1072 - acc: 0.9710 - val_loss: 0.2337 - val_acc: 0.9459
Epoch 8/10
60000/60000 [==============================] - 4s 

In [62]:
# .... yeah.. I think we are definitely overfitting now

## Different (possibly, random) architechtures

In [63]:
model_2 = Sequential()
model_2.add(Lambda(normalizer, input_shape=(1, 28, 28)))
model_2.add(Flatten())
model_2.add(Dense(512, activation='softmax'))
model_2.add(Dense(256, activation='softmax'))
model_2.add(Dense(10, activation='softmax'))
model_2.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'] )

In [64]:
model_2.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=10)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s - loss: 2.0548 - acc: 0.6553 - val_loss: 1.5061 - val_acc: 0.9267
Epoch 2/10
60000/60000 [==============================] - 5s - loss: 0.9280 - acc: 0.9260 - val_loss: 0.5481 - val_acc: 0.9300
Epoch 3/10
60000/60000 [==============================] - 5s - loss: 0.4294 - acc: 0.9342 - val_loss: 0.3718 - val_acc: 0.9335
Epoch 4/10
60000/60000 [==============================] - 5s - loss: 0.3307 - acc: 0.9377 - val_loss: 0.3345 - val_acc: 0.9316
Epoch 5/10
60000/60000 [==============================] - 5s - loss: 0.2890 - acc: 0.9414 - val_loss: 0.2983 - val_acc: 0.9383
Epoch 6/10
60000/60000 [==============================] - 5s - loss: 0.2701 - acc: 0.9434 - val_loss: 0.2953 - val_acc: 0.9346
Epoch 7/10
60000/60000 [==============================] - 5s - loss: 0.2548 - acc: 0.9451 - val_loss: 0.2872 - val_acc: 0.9369
Epoch 8/10
60000/60000 [==============================] - 5s 

In [66]:
model_2.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s - loss: 0.2143 - acc: 0.9509 - val_loss: 0.2537 - val_acc: 0.9400
Epoch 2/5
60000/60000 [==============================] - 5s - loss: 0.2056 - acc: 0.9522 - val_loss: 0.2440 - val_acc: 0.9447
Epoch 3/5
60000/60000 [==============================] - 5s - loss: 0.1987 - acc: 0.9537 - val_loss: 0.2448 - val_acc: 0.9434
Epoch 4/5
60000/60000 [==============================] - 5s - loss: 0.1885 - acc: 0.9550 - val_loss: 0.2502 - val_acc: 0.9407
Epoch 5/5
60000/60000 [==============================] - 5s - loss: 0.1817 - acc: 0.9560 - val_loss: 0.2424 - val_acc: 0.9432


In [67]:
# possible checkpoint to stop optimizing. can overfit

In [68]:
# let's reduce lr and continue

In [69]:
model_2.optimizer.lr = 0.0001

In [70]:
model_2.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=15)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 5s - loss: 0.1736 - acc: 0.9582 - val_loss: 0.2328 - val_acc: 0.9440
Epoch 2/15
60000/60000 [==============================] - 5s - loss: 0.1686 - acc: 0.9590 - val_loss: 0.2287 - val_acc: 0.9462
Epoch 3/15
60000/60000 [==============================] - 5s - loss: 0.1644 - acc: 0.9602 - val_loss: 0.2191 - val_acc: 0.9475
Epoch 4/15
60000/60000 [==============================] - 5s - loss: 0.1602 - acc: 0.9607 - val_loss: 0.2183 - val_acc: 0.9480
Epoch 5/15
60000/60000 [==============================] - 5s - loss: 0.1555 - acc: 0.9620 - val_loss: 0.2209 - val_acc: 0.9477
Epoch 6/15
60000/60000 [==============================] - 5s - loss: 0.1535 - acc: 0.9625 - val_loss: 0.2212 - val_acc: 0.9471
Epoch 7/15
60000/60000 [==============================] - 5s - loss: 0.1475 - acc: 0.9640 - val_loss: 0.2178 - val_acc: 0.9471
Epoch 8/15
60000/60000 [==============================] - 5s 

In [71]:
model_2.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s - loss: 0.1245 - acc: 0.9694 - val_loss: 0.2165 - val_acc: 0.9503
Epoch 2/5
60000/60000 [==============================] - 5s - loss: 0.1237 - acc: 0.9694 - val_loss: 0.2128 - val_acc: 0.9518
Epoch 3/5
60000/60000 [==============================] - 5s - loss: 0.1215 - acc: 0.9698 - val_loss: 0.2027 - val_acc: 0.9546
Epoch 4/5
60000/60000 [==============================] - 5s - loss: 0.1194 - acc: 0.9703 - val_loss: 0.2154 - val_acc: 0.9528
Epoch 5/5
60000/60000 [==============================] - 5s - loss: 0.1195 - acc: 0.9703 - val_loss: 0.2157 - val_acc: 0.9528


In [72]:
model_2.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s - loss: 0.1180 - acc: 0.9705 - val_loss: 0.2085 - val_acc: 0.9544
Epoch 2/5
60000/60000 [==============================] - 5s - loss: 0.1146 - acc: 0.9715 - val_loss: 0.2144 - val_acc: 0.9514
Epoch 3/5
60000/60000 [==============================] - 5s - loss: 0.1160 - acc: 0.9712 - val_loss: 0.2132 - val_acc: 0.9519
Epoch 4/5
60000/60000 [==============================] - 5s - loss: 0.1123 - acc: 0.9713 - val_loss: 0.2256 - val_acc: 0.9485
Epoch 5/5
60000/60000 [==============================] - 5s - loss: 0.1109 - acc: 0.9722 - val_loss: 0.2240 - val_acc: 0.9504


In [73]:
model_2.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=15)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 5s - loss: 0.1078 - acc: 0.9729 - val_loss: 0.2121 - val_acc: 0.9527
Epoch 2/15
60000/60000 [==============================] - 5s - loss: 0.1076 - acc: 0.9733 - val_loss: 0.2244 - val_acc: 0.9522
Epoch 3/15
60000/60000 [==============================] - 5s - loss: 0.1070 - acc: 0.9732 - val_loss: 0.2088 - val_acc: 0.9530
Epoch 4/15
60000/60000 [==============================] - 5s - loss: 0.1062 - acc: 0.9735 - val_loss: 0.2120 - val_acc: 0.9539
Epoch 5/15
60000/60000 [==============================] - 5s - loss: 0.1040 - acc: 0.9736 - val_loss: 0.2201 - val_acc: 0.9532
Epoch 6/15
60000/60000 [==============================] - 5s - loss: 0.1040 - acc: 0.9737 - val_loss: 0.2215 - val_acc: 0.9515
Epoch 7/15
60000/60000 [==============================] - 5s - loss: 0.1013 - acc: 0.9748 - val_loss: 0.2088 - val_acc: 0.9553
Epoch 8/15
60000/60000 [==============================] - 5s 

In [74]:
model_2.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=15)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 5s - loss: 0.0926 - acc: 0.9771 - val_loss: 0.2178 - val_acc: 0.9548
Epoch 2/15
60000/60000 [==============================] - 5s - loss: 0.0935 - acc: 0.9765 - val_loss: 0.2137 - val_acc: 0.9552
Epoch 3/15
60000/60000 [==============================] - 5s - loss: 0.0909 - acc: 0.9773 - val_loss: 0.2205 - val_acc: 0.9541
Epoch 4/15
60000/60000 [==============================] - 5s - loss: 0.0909 - acc: 0.9773 - val_loss: 0.2166 - val_acc: 0.9551
Epoch 5/15
60000/60000 [==============================] - 5s - loss: 0.0882 - acc: 0.9782 - val_loss: 0.2194 - val_acc: 0.9534
Epoch 6/15
60000/60000 [==============================] - 5s - loss: 0.0891 - acc: 0.9778 - val_loss: 0.2182 - val_acc: 0.9552
Epoch 7/15
60000/60000 [==============================] - 5s - loss: 0.0858 - acc: 0.9792 - val_loss: 0.2246 - val_acc: 0.9532
Epoch 8/15
60000/60000 [==============================] - 5s 

In [75]:
# Nope. We should've stopped the first time we hit around 95.5 val accuracy

## Next Architecture

In [76]:
model_3 = Sequential()
model_3.add(Lambda(normalizer, input_shape=(1, 28, 28)))
model_3.add(Flatten())
model_3.add(Dense(1024, activation='softmax'))
model_3.add(Dense(512, activation='softmax'))
model_3.add(Dense(256, activation='softmax'))
model_3.add(Dense(10, activation='softmax'))
model_3.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'] )

In [77]:
model_3.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 10s - loss: 2.2625 - acc: 0.1340 - val_loss: 2.0524 - val_acc: 0.2038
Epoch 2/10
60000/60000 [==============================] - 10s - loss: 1.8999 - acc: 0.2032 - val_loss: 1.8438 - val_acc: 0.2051
Epoch 3/10
60000/60000 [==============================] - 10s - loss: 1.8209 - acc: 0.2063 - val_loss: 1.7907 - val_acc: 0.2093
Epoch 4/10
60000/60000 [==============================] - 10s - loss: 1.6943 - acc: 0.3011 - val_loss: 1.6125 - val_acc: 0.3287
Epoch 5/10
60000/60000 [==============================] - 10s - loss: 1.5472 - acc: 0.3988 - val_loss: 1.4643 - val_acc: 0.4691
Epoch 6/10
60000/60000 [==============================] - 10s - loss: 1.3892 - acc: 0.4818 - val_loss: 1.3224 - val_acc: 0.4921
Epoch 7/10
60000/60000 [==============================] - 10s - loss: 1.2604 - acc: 0.4975 - val_loss: 1.1779 - val_acc: 0.4951
Epoch 8/10
60000/60000 [==============================

### The more complex the architecture is, more time it takes to fit it?

In [78]:
model_3.optimizer.lr = 0.01

In [79]:
model_3.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 10s - loss: 1.0536 - acc: 0.5587 - val_loss: 1.0626 - val_acc: 0.5737
Epoch 2/10
60000/60000 [==============================] - 10s - loss: 1.0121 - acc: 0.5800 - val_loss: 1.0260 - val_acc: 0.5803
Epoch 3/10
60000/60000 [==============================] - 10s - loss: 0.9850 - acc: 0.5837 - val_loss: 1.0075 - val_acc: 0.5822
Epoch 4/10
60000/60000 [==============================] - 10s - loss: 0.9672 - acc: 0.5853 - val_loss: 0.9847 - val_acc: 0.5811
Epoch 5/10
60000/60000 [==============================] - 10s - loss: 0.9534 - acc: 0.5871 - val_loss: 0.9754 - val_acc: 0.5866
Epoch 6/10
60000/60000 [==============================] - 10s - loss: 0.9451 - acc: 0.5895 - val_loss: 0.9829 - val_acc: 0.5793
Epoch 7/10
60000/60000 [==============================] - 10s - loss: 0.9397 - acc: 0.5915 - val_loss: 0.9775 - val_acc: 0.5830
Epoch 8/10
60000/60000 [==============================

In [80]:
model_3.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 10s - loss: 0.7788 - acc: 0.6699 - val_loss: 0.7911 - val_acc: 0.6657
Epoch 2/5
60000/60000 [==============================] - 10s - loss: 0.7510 - acc: 0.6729 - val_loss: 0.7813 - val_acc: 0.6628
Epoch 3/5
60000/60000 [==============================] - 10s - loss: 0.7342 - acc: 0.6767 - val_loss: 0.7737 - val_acc: 0.6688
Epoch 4/5
60000/60000 [==============================] - 10s - loss: 0.7238 - acc: 0.6779 - val_loss: 0.7759 - val_acc: 0.6656
Epoch 5/5
60000/60000 [==============================] - 10s - loss: 0.7189 - acc: 0.6799 - val_loss: 0.7629 - val_acc: 0.6714


In [81]:
model_3.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=15)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 10s - loss: 0.7122 - acc: 0.6815 - val_loss: 0.7608 - val_acc: 0.6691
Epoch 2/15
60000/60000 [==============================] - 10s - loss: 0.7049 - acc: 0.6877 - val_loss: 0.7573 - val_acc: 0.6766
Epoch 3/15
60000/60000 [==============================] - 10s - loss: 0.6976 - acc: 0.7043 - val_loss: 0.7230 - val_acc: 0.7371
Epoch 4/15
60000/60000 [==============================] - 10s - loss: 0.6129 - acc: 0.7633 - val_loss: 0.6233 - val_acc: 0.7635
Epoch 5/15
60000/60000 [==============================] - 10s - loss: 0.5530 - acc: 0.7739 - val_loss: 0.6044 - val_acc: 0.7680
Epoch 6/15
60000/60000 [==============================] - 10s - loss: 0.5264 - acc: 0.7782 - val_loss: 0.5735 - val_acc: 0.7680
Epoch 7/15
60000/60000 [==============================] - 10s - loss: 0.5092 - acc: 0.7837 - val_loss: 0.5621 - val_acc: 0.7747
Epoch 8/15
60000/60000 [==============================

In [82]:
## Big progess! What happened there? 

In [83]:
model_3.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=15)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 10s - loss: 0.2260 - acc: 0.9503 - val_loss: 0.2749 - val_acc: 0.9429
Epoch 2/15
60000/60000 [==============================] - 10s - loss: 0.2073 - acc: 0.9547 - val_loss: 0.2605 - val_acc: 0.9451
Epoch 3/15
60000/60000 [==============================] - 10s - loss: 0.1938 - acc: 0.9576 - val_loss: 0.2766 - val_acc: 0.9408
Epoch 4/15
60000/60000 [==============================] - 10s - loss: 0.1822 - acc: 0.9600 - val_loss: 0.2766 - val_acc: 0.9379
Epoch 5/15
60000/60000 [==============================] - 10s - loss: 0.1728 - acc: 0.9619 - val_loss: 0.2468 - val_acc: 0.9486
Epoch 6/15
60000/60000 [==============================] - 10s - loss: 0.1679 - acc: 0.9632 - val_loss: 0.2425 - val_acc: 0.9491
Epoch 7/15
60000/60000 [==============================] - 10s - loss: 0.1633 - acc: 0.9639 - val_loss: 0.2418 - val_acc: 0.9479
Epoch 8/15
60000/60000 [==============================

In [84]:
#took more time, but we're got to that 95.5~ mark. Let's go further

In [85]:
model_3.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 10s - loss: 0.1302 - acc: 0.9712 - val_loss: 0.2343 - val_acc: 0.9515
Epoch 2/5
60000/60000 [==============================] - 10s - loss: 0.1289 - acc: 0.9719 - val_loss: 0.2141 - val_acc: 0.9563
Epoch 3/5
60000/60000 [==============================] - 10s - loss: 0.1275 - acc: 0.9727 - val_loss: 0.2223 - val_acc: 0.9545
Epoch 4/5
60000/60000 [==============================] - 10s - loss: 0.1242 - acc: 0.9728 - val_loss: 0.2437 - val_acc: 0.9516
Epoch 5/5
60000/60000 [==============================] - 10s - loss: 0.1213 - acc: 0.9740 - val_loss: 0.2340 - val_acc: 0.9506


In [86]:
model_3.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=10)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 10s - loss: 0.1225 - acc: 0.9732 - val_loss: 0.2214 - val_acc: 0.9555
Epoch 2/10
60000/60000 [==============================] - 10s - loss: 0.1181 - acc: 0.9739 - val_loss: 0.2264 - val_acc: 0.9539
Epoch 3/10
60000/60000 [==============================] - 10s - loss: 0.1183 - acc: 0.9738 - val_loss: 0.2195 - val_acc: 0.9556
Epoch 4/10
60000/60000 [==============================] - 10s - loss: 0.1158 - acc: 0.9744 - val_loss: 0.2235 - val_acc: 0.9539
Epoch 5/10
60000/60000 [==============================] - 10s - loss: 0.1144 - acc: 0.9751 - val_loss: 0.2261 - val_acc: 0.9543
Epoch 6/10
60000/60000 [==============================] - 10s - loss: 0.1134 - acc: 0.9754 - val_loss: 0.2226 - val_acc: 0.9562
Epoch 7/10
60000/60000 [==============================] - 10s - loss: 0.1124 - acc: 0.9754 - val_loss: 0.2363 - val_acc: 0.9541
Epoch 8/10
60000/60000 [==============================

In [87]:
model_3.optimizer.lr = 0.001

In [88]:
model_3.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=15)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 10s - loss: 0.1092 - acc: 0.9761 - val_loss: 0.2283 - val_acc: 0.9536
Epoch 2/15
60000/60000 [==============================] - 10s - loss: 0.1047 - acc: 0.9772 - val_loss: 0.2309 - val_acc: 0.9531
Epoch 3/15
60000/60000 [==============================] - 10s - loss: 0.1064 - acc: 0.9765 - val_loss: 0.2304 - val_acc: 0.9542
Epoch 4/15
60000/60000 [==============================] - 10s - loss: 0.1061 - acc: 0.9768 - val_loss: 0.2506 - val_acc: 0.9502
Epoch 5/15
60000/60000 [==============================] - 10s - loss: 0.1040 - acc: 0.9773 - val_loss: 0.2321 - val_acc: 0.9549
Epoch 6/15
60000/60000 [==============================] - 10s - loss: 0.1013 - acc: 0.9775 - val_loss: 0.2327 - val_acc: 0.9538
Epoch 7/15
60000/60000 [==============================] - 10s - loss: 0.1005 - acc: 0.9780 - val_loss: 0.2274 - val_acc: 0.9567
Epoch 8/15
60000/60000 [==============================

In [89]:
model_3.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=15)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 10s - loss: 0.0941 - acc: 0.9795 - val_loss: 0.2452 - val_acc: 0.9519
Epoch 2/15
60000/60000 [==============================] - 10s - loss: 0.0946 - acc: 0.9790 - val_loss: 0.2396 - val_acc: 0.9544
Epoch 3/15
60000/60000 [==============================] - 10s - loss: 0.0916 - acc: 0.9800 - val_loss: 0.2368 - val_acc: 0.9543
Epoch 4/15
60000/60000 [==============================] - 10s - loss: 0.0907 - acc: 0.9798 - val_loss: 0.2322 - val_acc: 0.9552
Epoch 5/15
60000/60000 [==============================] - 10s - loss: 0.0905 - acc: 0.9801 - val_loss: 0.2384 - val_acc: 0.9531
Epoch 6/15
60000/60000 [==============================] - 10s - loss: 0.0925 - acc: 0.9798 - val_loss: 0.2341 - val_acc: 0.9552
Epoch 7/15
60000/60000 [==============================] - 10s - loss: 0.0899 - acc: 0.9806 - val_loss: 0.2434 - val_acc: 0.9528
Epoch 8/15
60000/60000 [==============================

In [90]:
model_3.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 10s - loss: 0.0865 - acc: 0.9808 - val_loss: 0.2401 - val_acc: 0.9531
Epoch 2/5
60000/60000 [==============================] - 10s - loss: 0.0846 - acc: 0.9813 - val_loss: 0.2450 - val_acc: 0.9539
Epoch 3/5
60000/60000 [==============================] - 10s - loss: 0.0839 - acc: 0.9813 - val_loss: 0.2500 - val_acc: 0.9534
Epoch 4/5
60000/60000 [==============================] - 10s - loss: 0.0846 - acc: 0.9812 - val_loss: 0.2393 - val_acc: 0.9537
Epoch 5/5
60000/60000 [==============================] - 10s - loss: 0.0834 - acc: 0.9817 - val_loss: 0.2423 - val_acc: 0.9528


In [91]:
model_3.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), nb_epoch=15)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 10s - loss: 0.0831 - acc: 0.9821 - val_loss: 0.2457 - val_acc: 0.9547
Epoch 2/15
60000/60000 [==============================] - 10s - loss: 0.0835 - acc: 0.9817 - val_loss: 0.2361 - val_acc: 0.9546
Epoch 3/15
60000/60000 [==============================] - 10s - loss: 0.0811 - acc: 0.9826 - val_loss: 0.2454 - val_acc: 0.9539
Epoch 4/15
60000/60000 [==============================] - 10s - loss: 0.0827 - acc: 0.9822 - val_loss: 0.2418 - val_acc: 0.9552
Epoch 5/15
60000/60000 [==============================] - 10s - loss: 0.0820 - acc: 0.9814 - val_loss: 0.2326 - val_acc: 0.9559
Epoch 6/15
60000/60000 [==============================] - 10s - loss: 0.0799 - acc: 0.9826 - val_loss: 0.2435 - val_acc: 0.9537
Epoch 7/15
60000/60000 [==============================] - 10s - loss: 0.0804 - acc: 0.9824 - val_loss: 0.2380 - val_acc: 0.9543
Epoch 8/15
60000/60000 [==============================